# Installing the Packages

In [ ]:
!pip install scikit-image

# Importing the Packages

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread

import pickle

In [ ]:
new_image_size = (150,150,3)

# Loading the Training Images

In [ ]:
import os
import imageio
import matplotlib.pyplot as plt

# set the directory containing the images
images_dir = './Fruits'

i = 0
current_id = 0

# for storing the foldername: label, 
# e.g. {'banana': 0, 'durian': 1, 'orange': 2, 'strawberry': 3}
label_ids = {}

# for storing the images data and labels
images = []
labels = []

for root, _, files in os.walk(images_dir):
    for file in files:
        if file.endswith(('png','jpg','jpeg')):
            # path of the image
            path = os.path.join(root, file)
            # print(path)            
            # get the label name
            label = os.path.basename(root).replace(" ", ".").lower()                        
            
            # add the label (key) and its number (value)
            if not label in label_ids:
                label_ids[label] = current_id                
                current_id += 1 
            
            # save the target value    
            labels.append(current_id-1)
            
            # load the image, resize and flatten it
            image = imread(path)
            image = resize(image, new_image_size)
            images.append(image.flatten())        
            
            # show the image
            plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
            plt.show()            

print(label_ids)
# save the labels for each fruit as a list            
categories = list(label_ids.keys())

pickle.dump(categories, open( "fruit_labels.pk", "wb" ) )

df = pd.DataFrame(np.array(images))
df['Target'] = np.array(labels)
df.sample(5)

In [ ]:
df.sample(5)

# Splitting the Data into Training and Testing Sets

In [ ]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.10,
                                                    random_state=77,
                                                    stratify = y)

# Finding and Training the Best Model

In [ ]:
# trying out the various parameters
param_grid = {
    'C'      : [0.1, 1, 10, 100],
    'gamma'  : [0.0001, 0.001, 0.1, 1], 
    'kernel' : ['rbf', 'poly']
}

svc = svm.SVC(probability=True)
print("Starting training, please wait ...")

# exhaustive search over specified parameter values for an estimator
model = GridSearchCV(svc,param_grid)
model.fit(x_train, y_train)

# print the parameters for the best performing model
print('The model has been trained with the given images')
model.best_params_

# Predicting using the Testing Set

In [ ]:
y_pred = model.predict(x_test)
print("Predicted Data:")
y_pred

In [ ]:
print("Actual Data:")
np.array(y_test)

In [ ]:
print(f"The model is {accuracy_score(y_pred,y_test) * 100}% accurate")

print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

# Saving the Model using Pickle

In [ ]:
pickle.dump(model, open('fruits_model.pickle','wb'))


# Testing the Model

In [ ]:
# loading the model
model      = pickle.load(open('fruits_model.pickle','rb'))
categories = pickle.load(open('fruit_labels.pk', 'rb'))


for i in range(1,20):    # 20 for fruits; 68 for faces
    # load the test image
    url = f'./fruittest/fruit{i}.jpg'
    img = imread(url)
    plt.imshow(img)
    plt.show()

    # resize and flatten the image
    img_resize = resize(img, new_image_size)
    flattened_image = [img_resize.flatten()]

    # predict the probability
    probability = model.predict_proba(flattened_image)

    for i, val in enumerate(categories):
        print(f'{val} = {probability[0][i] * 100}%')

    print('***********************************************')
    print(f"Predicted image is: {categories[model.predict(flattened_image)[0]]}")
    print('***********************************************')
